In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
benchmark_folder = Path('../../benchmark_haddock_27_July_2024')

In [3]:
plddt_antigen_df = pd.read_csv('../data/AF2_antigen_rmsd_plddt_multi_regions.csv')
plddt_antibody_df = pd.read_csv('../data/AF2_antibody_rmsd_plddt_multi_regions.csv')
pdbcodes = plddt_antibody_df['pdb'].unique()
interface_plddts = []
for pdbcode in pdbcodes:
    paratope_res = set()
    epitope_res = set()
    with (benchmark_folder/f'{pdbcode}/{pdbcode}_constraint_pairs.txt').open('r') as file:
        for line in file.readlines()[1:]:
            antibody_line, antigen_line = line.split(':')
            paratope_res.add(''.join(antibody_line.split(',')[:2]))
            epitope_res.add(''.join(antigen_line.split(',')[:2]))
    n_paratope_res = len(paratope_res)
    n_epitope_res = len(epitope_res)
    n_interface_res = n_paratope_res+n_epitope_res
    epitope_plddt = plddt_antigen_df.query('pdb==@pdbcode')['plddt_ave_para_epi'].to_list()[0]
    paratope_plddt = plddt_antibody_df.query('pdb==@pdbcode')['plddt_ave_para_epi'].to_list()[0]
    interface_plddt = (epitope_plddt*n_epitope_res + paratope_plddt*n_paratope_res)/n_interface_res
    interface_plddts.append(interface_plddt)

interface_plddt_df = pd.DataFrame.from_dict({'pdb': pdbcodes, 'interface_ave_plddt': interface_plddts})

In [4]:
interface_plddt_df.to_csv('../data/AF2_interface_plddt.csv')